In [1]:
import sys
!{sys.executable} -m pip install keras tensorflow --upgrade
!{sys.executable} -m pip install keras_tuner

import os
import joblib
import numpy as np
import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from keras import backend as K
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
DATA_PATH = "../data/raw/training_data.npz"
SCALER_DIR = "../data/processed/scalers"
MODEL_DIR = "../src/models"
FIGURE_DIR = "../src/visualization/plots"
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(FIGURE_DIR, exist_ok=True)

In [3]:
def load_dataset(path=DATA_PATH):
    """Load dataset from NPZ file."""
    data = np.load(path)
    X, y = data["X"], data["y"]
    print(f"Loaded dataset: X shape {X.shape}, y shape {y.shape}")
    return X, y

X, y = load_dataset()

Loaded dataset: X shape (450000, 15), y shape (450000,)


In [4]:
def preprocess_data(X, y, test_size=0.2, random_state=42):
    """
    Splits into train/test and normalizes features.
    Prefix prices + numeric features are scaled.
    opt_flag is kept as-is (categorical 0/1).
    """
    # Split train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    # Separate opt_flag (last column)
    X_train_prefix = X_train[:, :-1]
    X_test_prefix = X_test[:, :-1]

    opt_flag_train = X_train[:, -1].reshape(-1, 1)
    opt_flag_test = X_test[:, -1].reshape(-1, 1)

    # Scale everything except opt_flag
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_prefix)
    X_test_scaled = scaler.transform(X_test_prefix)

    # Reattach opt_flag
    X_train_final = np.hstack([X_train_scaled, opt_flag_train])
    X_test_final = np.hstack([X_test_scaled, opt_flag_test])

    # Save scaler for later inference
    os.makedirs(SCALER_DIR, exist_ok=True)
    joblib.dump(scaler, os.path.join(SCALER_DIR, "feature_scaler.pkl"))

    print(f"Train set: {X_train_final.shape}, Test set: {X_test_final.shape}")
    return X_train_final, X_test_final, y_train, y_test

X_train, X_test, y_train, y_test = preprocess_data(X, y)

Train set: (360000, 15), Test set: (90000, 15)


In [5]:
input_dim = X_train.shape[1]
input_dim

15

In [6]:
def plot_training(history, title, filename):
    """Save training curves for loss + MAE + RMSE."""
    plt.figure(figsize=(15, 4))

    # Loss plot
    plt.subplot(1, 3, 1)
    plt.plot(history.history["loss"], label="Train Loss")
    plt.plot(history.history["val_loss"], label="Val Loss")
    plt.title(f"{title} - Loss")
    plt.xlabel("Epoch")
    plt.ylabel("MSE Loss")
    plt.legend()

    # MAE plot
    plt.subplot(1, 3, 2)
    plt.plot(history.history["mae"], label="Train MAE")
    plt.plot(history.history["val_mae"], label="Val MAE")
    plt.title(f"{title} - MAE")
    plt.xlabel("Epoch")
    plt.ylabel("Mean Absolute Error")
    plt.legend()

    # RMSE plot (computed from loss)
    train_rmse = np.sqrt(history.history["loss"])
    val_rmse = np.sqrt(history.history["val_loss"])
    plt.subplot(1, 3, 3)
    plt.plot(train_rmse, label="Train RMSE")
    plt.plot(val_rmse, label="Val RMSE")
    plt.title(f"{title} - RMSE")
    plt.xlabel("Epoch")
    plt.ylabel("Root MSE")
    plt.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(FIGURE_DIR, filename))
    plt.close()


In [7]:
def build_quick_mlp(input_dim):
    """Small, quick baseline MLP."""
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(128, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(1, activation="linear")
    ])
    model.compile(optimizer="adam", loss="mse", metrics=["mae"])
    return model


def evaluate_model(model, X_test, y_test):
    """Compute MAE and RMSE on test set."""
    y_pred = model.predict(X_test)
    mae = np.mean(np.abs(y_test - y_pred.flatten()))
    rmse = np.sqrt(np.mean((y_test - y_pred.flatten()) ** 2))
    print(f"Evaluation on Test Set -> MAE: {mae:.4f}, RMSE: {rmse:.4f}")
    return mae, rmse

quick_mlp = build_quick_mlp(input_dim)
history_quick = quick_mlp.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=64,
    verbose=1
)
quick_mlp.save(os.path.join(MODEL_DIR, "mlp_quick.h5"))
plot_training(history_quick, "Quick MLP", "training_quick_run_2.png")
evaluate_model(quick_mlp, X_test, y_test)

Epoch 1/10
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 5s 765us/step - loss: 95.7995 - mae: 6.1141 - val_loss: 91.7610 - val_mae: 5.8399
Epoch 2/10
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 4s 743us/step - loss: 91.3910 - mae: 5.8572 - val_loss: 91.8453 - val_mae: 5.8321
Epoch 3/10
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 4s 745us/step - loss: 91.1803 - mae: 5.8401 - val_loss: 92.0253 - val_mae: 5.7847
Epoch 4/10
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 4s 744us/step - loss: 91.0460 - mae: 5.8312 - val_loss: 91.6385 - val_mae: 5.8261
Epoch 5/10
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 4s 745us/step - loss: 91.0170 - mae: 5.8271 - val_loss: 91.2610 - val_mae: 5.8690
Epoch 6/10
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 4s 776us/step - loss: 90.9624 - mae: 5.8224 - val_loss: 91.3123 - val_mae: 5.8537
Epoch 7/10
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 4s 763us/step - loss: 90.9252 - mae: 5.8189 - val_loss: 91.8168 - val_mae: 5.8414
Epoch 8/10
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 5s 800us/step - loss: 90.8532 - mae: 5.8174 - val_loss: 91.9342 - val_mae: 5.7876
Epoch 9/

2813/2813 ━━━━━━━━━━━━━━━━━━━━ 1s 392us/step
Evaluation on Test Set -> MAE: 5.8552, RMSE: 9.5600


(np.float64(5.855225371268371), np.float64(9.56002060306431))

In [8]:
def build_large_mlp(hp):
    model = keras.Sequential()
    input_dim = X_train.shape[1]

    # Input layer
    model.add(layers.Input(shape=(input_dim,)))

    # Number of hidden layers
    num_layers = hp.Int("num_layers", min_value=2, max_value=6, step=1)

    for i in range(num_layers):
        units = hp.Int(f"units_{i}", min_value=32, max_value=512, step=32)
        activation = hp.Choice("activation", ["relu", "tanh"])
        model.add(layers.Dense(units=units, activation=activation))

    # Output layer
    model.add(layers.Dense(1, activation="linear"))

    # Optimizer
    optimizer_choice = hp.Choice("optimizer", ["adam", "sgd"])
    learning_rate = hp.Float("lr", 1e-4, 1e-2, sampling="log")

    if optimizer_choice == "adam":
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)

    model.compile(
        optimizer=optimizer,
        loss="mse",
        metrics=["mae", "mse", "accuracy"]  # include accuracy for monitoring
    )
    return model


In [9]:
# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Tuner
tuner = kt.Hyperband(
    build_large_mlp,
    objective="val_mae",
    max_epochs=50,  # upper bound for epochs
    factor=3,
    directory="tuner_logs",
    project_name="large_mlp_tuning"
)

stop_early = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)
tuner.search(
    X_train, y_train,
    validation_data=(X_test, y_test),
    callbacks=[stop_early],
    batch_size=kt.HyperParameters().Int("batch_size", min_value=32, max_value=256, step=32),
    epochs=50
)
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
for hp in best_hps.values.keys():
    print(f"  - {hp}: {best_hps.get(hp)}")

Reloading Tuner from tuner_logs\large_mlp_tuning\tuner0.json
  - num_layers: 3
  - units_0: 96
  - activation: relu
  - units_1: 192
  - optimizer: sgd
  - lr: 0.0005822751269209618
  - units_2: 352
  - units_3: 320
  - units_4: 384
  - units_5: 352
  - tuner/epochs: 6
  - tuner/initial_epoch: 0
  - tuner/bracket: 2
  - tuner/round: 0


In [10]:
model = tuner.hypermodel.build(best_hps)

batch_size = 64
epochs = 50


history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1
)

Epoch 1/50
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.1506 - loss: 95.1052 - mae: 5.9999 - mse: 95.1052 - val_accuracy: 0.1557 - val_loss: 93.3885 - val_mae: 6.0633 - val_mse: 93.3885
Epoch 2/50
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 6s 999us/step - accuracy: 0.1744 - loss: 91.7001 - mae: 5.8265 - mse: 91.7001 - val_accuracy: 0.1596 - val_loss: 91.6718 - val_mae: 5.9242 - val_mse: 91.6718
Epoch 3/50
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.1756 - loss: 91.4042 - mae: 5.8137 - mse: 91.4042 - val_accuracy: 0.1588 - val_loss: 91.5821 - val_mae: 5.8260 - val_mse: 91.5821
Epoch 4/50
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.1740 - loss: 91.2848 - mae: 5.8116 - mse: 91.2848 - val_accuracy: 0.1635 - val_loss: 91.4617 - val_mae: 5.8691 - val_mse: 91.4617
Epoch 5/50
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.1743 - loss: 91.1743 - mae: 5.8087 - mse: 91.1743 - val_accuracy: 0.1635 - val_loss: 91.2359 - val_mae: 5.8649 - val_mse: 91.2359
Epoch 6/

In [11]:
y_pred = model.predict(X_test).flatten()
rmse = np.sqrt(np.mean((y_test - y_pred) ** 2))
mae = np.mean(np.abs(y_test - y_pred))
acc = np.mean(np.isclose(np.round(y_test), np.round(y_pred)))  # crude regression accuracy

print(f"\nFinal Model Evaluation:\n  MAE: {mae:.4f}\n  RMSE: {rmse:.4f}\n  Accuracy: {acc:.4f}")
model.save(os.path.join(MODEL_DIR, "mlp_large_tuned.h5"))
plot_training(history, "Large MLP (Tuned)", "training_large_tuned.png")

2813/2813 ━━━━━━━━━━━━━━━━━━━━ 1s 466us/step



Final Model Evaluation:
  MAE: 5.9689
  RMSE: 9.6586
  Accuracy: 0.1652


Not the best numbers, next few steps - 
1. Baseline from mean of y_train -> check wif the NN is beating this
2. Try different batch size and epochs

In [12]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

y_baseline = np.full_like(y_test_scaled, fill_value=np.mean(y_train_scaled))
baseline_mae = mean_absolute_error(y_test_scaled, y_baseline)
baseline_rmse = np.sqrt(mean_squared_error(y_test_scaled, y_baseline))
baseline_r2 = r2_score(y_test_scaled, y_baseline)

print(f"\nBaseline (mean predictor):\n"
      f"  MAE: {baseline_mae:.4f}\n"
      f"  RMSE: {baseline_rmse:.4f}\n"
      f"  R²: {baseline_r2:.4f}")


Baseline (mean predictor):
  MAE: 0.7706
  RMSE: 1.0066
  R²: -0.0001


In [13]:
model = tuner.hypermodel.build(best_hps)

In [14]:
history = model.fit(
    X_train_scaled, y_train_scaled,
    validation_data=(X_test_scaled, y_test_scaled),
    epochs=25,
    batch_size=128,
    verbose=1,
)

Epoch 1/25


2813/2813 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.0000e+00 - loss: 0.8933 - mae: 0.7242 - mse: 0.8933 - val_accuracy: 0.0000e+00 - val_loss: 0.8036 - val_mae: 0.6713 - val_mse: 0.8036
Epoch 2/25
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.0000e+00 - loss: 0.6995 - mae: 0.5961 - mse: 0.6995 - val_accuracy: 0.0000e+00 - val_loss: 0.6329 - val_mae: 0.5392 - val_mse: 0.6329
Epoch 3/25
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.0000e+00 - loss: 0.6030 - mae: 0.5137 - mse: 0.6030 - val_accuracy: 0.0000e+00 - val_loss: 0.5925 - val_mae: 0.5060 - val_mse: 0.5925
Epoch 4/25
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.0000e+00 - loss: 0.5827 - mae: 0.4941 - mse: 0.5827 - val_accuracy: 0.0000e+00 - val_loss: 0.5817 - val_mae: 0.4916 - val_mse: 0.5817
Epoch 5/25
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.0000e+00 - loss: 0.5748 - mae: 0.4844 - mse: 0.5748 - val_accuracy: 0.0000e+00 - val_loss: 0.5761 - val_mae: 0.4832 - val_mse: 0.5761
E

In [15]:
y_pred_scaled = model.predict(X_test_scaled).flatten()
mae = mean_absolute_error(y_test_scaled, y_pred_scaled)
rmse = np.sqrt(mean_squared_error(y_test_scaled, y_pred_scaled))
r2 = r2_score(y_test_scaled, y_pred_scaled)

print(f"\nFinal Model Evaluation (scaled):\n"
      f"  MAE: {mae:.4f}\n"
      f"  RMSE: {rmse:.4f}\n"
      f"  R²: {r2:.4f}")

2813/2813 ━━━━━━━━━━━━━━━━━━━━ 1s 435us/step

Final Model Evaluation (scaled):
  MAE: 0.4613
  RMSE: 0.7479
  R²: 0.4479


- Model seems to be learning from data, with drops in RMSE and MAE
- Still improvements to be made
- Explore feature engineering, more complex models regularization, CV?

## Further Improvements: Feature Engineering & Advanced Tuning
We'll now try to improve the large MLP by adding feature engineering and using more advanced hyperparameter search methods.

In [16]:
# Feature Engineering: Add polynomial and interaction features
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
X_train_fe = poly.fit_transform(X_train)
X_test_fe = poly.transform(X_test)

print(f"Feature engineered shapes: X_train_fe {X_train_fe.shape}, X_test_fe {X_test_fe.shape}")

Feature engineered shapes: X_train_fe (360000, 135), X_test_fe (90000, 135)


In [ ]:
# Advanced Hyperparameter Tuning: Bayesian Optimization
from keras_tuner.tuners import BayesianOptimization

def build_advanced_mlp(hp):
    model = keras.Sequential()
    input_dim = X_train_fe.shape[1]
    model.add(layers.Input(shape=(input_dim,)))
    num_layers = hp.Int("num_layers", 2, 6)
    for i in range(num_layers):
        units = hp.Int(f"units_{i}", 32, 512, step=32)
        activation = hp.Choice(f"activation_{i}", ["relu", "tanh"])
        model.add(layers.Dense(units=units, activation=activation))
    model.add(layers.Dense(1, activation="linear"))
    optimizer_choice = hp.Choice("optimizer", ["adam", "sgd", "rmsprop"])
    learning_rate = hp.Float("lr", 1e-4, 1e-2, sampling="log")
    if optimizer_choice == "adam":
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_choice == "sgd":
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss="mse", metrics=["mae", "mse"])
    return model

bayes_tuner = BayesianOptimization(
    build_advanced_mlp,
    objective="val_mae",
    max_trials=20,
    directory="tuner_logs",
    project_name="large_mlp_bayes_tuning"
)

stop_early = keras.callbacks.EarlyStopping(monitor="val_loss", patience=7)
bayes_tuner.search(
    X_train_fe, y_train,
    validation_data=(X_test_fe, y_test),
    callbacks=[stop_early],
    batch_size=64,
    epochs=40
)
best_bayes_hps = bayes_tuner.get_best_hyperparameters(num_trials=1)[0]
for hp in best_bayes_hps.values.keys():
    print(f"  - {hp}: {best_bayes_hps.get(hp)}")

Trial 5 Complete [00h 01m 24s]
val_mae: 5.8582682609558105

Best val_mae So Far: 5.798402309417725
Total elapsed time: 00h 12m 17s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
4                 |4                 |num_layers
384               |352               |units_0
relu              |tanh              |activation_0
192               |448               |units_1
tanh              |tanh              |activation_1
adam              |adam              |optimizer
0.0016335         |0.00026556        |lr
480               |32                |units_2
relu              |relu              |activation_2
384               |32                |units_3
relu              |relu              |activation_3
288               |None              |units_4
relu              |None              |activation_4

Epoch 1/40
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 95.5278 - mae: 5.9638 - mse: 95.5278 - val_loss: 93.3207 - val_mae: 5.9749 - val_mse: 93.3207
Epoch 2/

In [ ]:
# Train and evaluate improved model
advanced_model = bayes_tuner.hypermodel.build(best_bayes_hps)
history_adv = advanced_model.fit(
    X_train_fe, y_train,
    validation_data=(X_test_fe, y_test),
    epochs=40,
    batch_size=64,
    verbose=1
)

# Evaluate
y_pred_adv = advanced_model.predict(X_test_fe).flatten()
mae_adv = mean_absolute_error(y_test, y_pred_adv)
rmse_adv = np.sqrt(mean_squared_error(y_test, y_pred_adv))
r2_adv = r2_score(y_test, y_pred_adv)

print(f"\nImproved Model Evaluation:\n  MAE: {mae_adv:.4f}\n  RMSE: {rmse_adv:.4f}\n  R²: {r2_adv:.4f}")
advanced_model.save(os.path.join(MODEL_DIR, "mlp_large_advanced_tuned.h5"))
plot_training(history_adv, "Large MLP (Advanced)", "training_large_advanced_tuned.png")

## Advanced Model Improvements
Let's try more advanced deep learning techniques: residual connections, dropout, batch normalization, and learning rate scheduling.

In [ ]:
# Residual MLP with Dropout and BatchNorm
from tensorflow.keras import regularizers

def build_residual_mlp(input_dim, n_layers=4, units=128, dropout_rate=0.2, l2_reg=1e-4):
    inputs = keras.Input(shape=(input_dim,))
    x = layers.Dense(units, activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(inputs)
    for i in range(n_layers):
        shortcut = x
        x = layers.Dense(units, activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.Add()([x, shortcut])  # Residual connection
    outputs = layers.Dense(1, activation='linear')(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=keras.optimizers.Adam(), loss='mse', metrics=['mae', 'mse'])
    return model

res_mlp = build_residual_mlp(X_train_fe.shape[1], n_layers=3, units=256, dropout_rate=0.3, l2_reg=1e-3)
lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)

history_res = res_mlp.fit(
    X_train_fe, y_train,
    validation_data=(X_test_fe, y_test),
    epochs=40,
    batch_size=64,
    callbacks=[lr_scheduler],
    verbose=1
)

# Evaluate
res_pred = res_mlp.predict(X_test_fe).flatten()
mae_res = mean_absolute_error(y_test, res_pred)
rmse_res = np.sqrt(mean_squared_error(y_test, res_pred))
r2_res = r2_score(y_test, res_pred)

print(f"\nResidual MLP Evaluation:\n  MAE: {mae_res:.4f}\n  RMSE: {rmse_res:.4f}\n  R²: {r2_res:.4f}")
res_mlp.save(os.path.join(MODEL_DIR, "mlp_large_residual_tuned.h5"))
plot_training(history_res, "Residual MLP", "training_residual_mlp.png")

In [ ]:
# Ensemble: Average predictions from best models
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Predict with advanced and residual models
ensemble_preds = (y_pred_adv + res_pred) / 2
mae_ensemble = mean_absolute_error(y_test, ensemble_preds)
rmse_ensemble = np.sqrt(mean_squared_error(y_test, ensemble_preds))
r2_ensemble = r2_score(y_test, ensemble_preds)

print(f"\nEnsemble Model Evaluation:\n  MAE: {mae_ensemble:.4f}\n  RMSE: {rmse_ensemble:.4f}\n  R²: {r2_ensemble:.4f}")